数据有关键词、每个数据带有特定类别？我们怎样可以快速找到对应的类别，聚类分析？

Pandas 中就有提供这样便捷的功能，groupby()。简单来说，groupby 就是提前帮你准备好快筛功能， 在你想要筛选特定数据的时候，快速帮你找到对应的数据块。 而且还可以针对不同 group 做不同的数据处理。

### 分组
对于 groupby() 最核心的功能，自然就是将可以被归纳的数据进行归纳汇总。

In [1]:
import pandas as pd

df = pd.DataFrame(
    [
        ("小红", "哈利波特", 80),
        ("小明", "蜘蛛侠", 72),
        ("小红", "雷神", 83),
        ("小红", "蜘蛛侠", 45),
        ("小明", "超人", 57),
    ],
    columns=("人", "人物", "评价"),
)
df

,人,人物,评价
0,小红,哈利波特,80
1,小明,蜘蛛侠,72
2,小红,雷神,83
3,小红,蜘蛛侠,45
4,小明,超人,57


比如我有上面这组数据，描述了每个人对人物的评价。这组数据的形成可能就是一段时间内，收集了不同人对各种人物的评价，所以它的组织顺序会比较乱。 好在我们还可以用 groupby 来重组。

In [2]:
grouped = df.groupby("人")
grouped

上面我们就获取了 Pandas 当中的 group 类，再看看这个 grouped 当中究竟是怎么组织我们的数据的。

In [4]:
grouped.groups #保存的其实是不同 人 的 row index

{'小明': [1, 4], '小红': [0, 2, 3]}

你会发现，它保存的其实是不同 人 的 row index, 这样后面重新在找 group 组的时候，可以快速用 index 找到对应行。 要直接获取不同的 人 组，我们看看怎么搞？

In [5]:
df.iloc[grouped.groups["小红"]]

,人,人物,评价
0,小红,哈利波特,80
2,小红,雷神,83
3,小红,蜘蛛侠,45


对，上面的确是一种我们用以前学的知识就能找到的办法，不过 懒人工具 pandas 怎么能让我们这么费力呢？ 所以它直接能在 grouped 的基础上，get_group()。

In [6]:
grouped.get_group("小红")

,人,人物,评价
0,小红,哈利波特,80
2,小红,雷神,83
3,小红,蜘蛛侠,45


你看，这上下两种方法其实可以达到一样的效果。

### 调用分好的组
除了 grouped.get_group()，其实还有更多方式来获取到分好的组，应对你不同的应用需求。 而这些不同的筛选方法，其实本质还是在对 grouped.groups 中的 index 进行操作的一个过程。 比如如果我要选第一个数据，pandas 就会调用 groups 里面的第一个 index。

In [7]:
grouped.first() # 返回给你每一个组的第一个数据

,人物,评价
人,,
小明,蜘蛛侠,72
小红,哈利波特,80


注意，它会返回给你每一个组的第一个数据，而如果我要找最后一个，也是同样的逻辑。

In [8]:
grouped.last()

,人物,评价
人,,
小明,超人,57
小红,蜘蛛侠,45


甚至你还可以做一些聚合操作，比如对每一个组里面的数据进行 sum, mean 等操作。你可以在下面的执行区都试试。

In [9]:
grouped.sum()

,评价
人,
小明,129
小红,208


### 循环处理
有时候你想要对组进行循环处理，通过一个循环最所有组统一操作一下。那你需要注意一下这个 grouped 的循环，会带着两个字段做循环。 一个是组名，一个是组数据。

In [10]:
for name, group in grouped:
    print("name:", name)
    print(group)

name: 小明
    人   人物  评价
1  小明  蜘蛛侠  72
4  小明   超人  57
name: 小红
    人    人物  评价
0  小红  哈利波特  80
2  小红    雷神  83
3  小红   蜘蛛侠  45


### 多从分组
数据真的很多，在不同分组后，我们还想对另一 column 的值也进行分组？ 比如小红今天再看了一遍雷神，觉得雷神今年的表现不错，对雷神做了一次新的高分评价。

In [11]:
df = pd.DataFrame(
    [
        ("小红", "哈利波特", 80),
        ("小明", "蜘蛛侠", 72),
        ("小红", "雷神", 83),
        ("小红", "雷神", 90),
        ("小红", "蜘蛛侠", 45),
        ("小明", "超人", 57),
    ],
    columns=("人", "人物", "评价"),
)
df

,人,人物,评价
0,小红,哈利波特,80
1,小明,蜘蛛侠,72
2,小红,雷神,83
3,小红,雷神,90
4,小红,蜘蛛侠,45
5,小明,超人,57


这时，小红数据中就会有两次雷神的评价了。如果这类的事情发生很多，我们也可以深入划分一下细分领域的组信息。 比如我按照 人 和 人物 进行分组，而且选择 小红 的 雷神 评分进行查看。

In [12]:
df.groupby(["人", "人物"]).get_group(("小红", "雷神"))

,人,人物,评价
2,小红,雷神,83
3,小红,雷神,90


### 聚合计算
对选出来的数据，有时候你并不仅仅只是为了用眼直接看，你还想要对筛选出来的数据进行统计学上的运算。比如上面我举的 grouped.sum() 这个例子。

In [13]:
import numpy as np

grouped = df.groupby("人")
grouped.aggregate(np.sum)

,评价
人,
小明,129
小红,298


如果你想要对数据一次性做更多的计算，你可以多加几个要计算的操作在后面，比如 [sum, mean, std]，注意下面的 .agg() 其实是 .aggregate() 的缩写， 它俩是一样的。下面我们就对每组的 评价 指标进行一次性的多种运算。

In [14]:
grouped["评价"].agg([np.sum, np.mean, np.std])

,sum,mean,std
人,,,
小明,129,64.5,10.606602
小红,298,74.5,20.108042


还有一个小技巧，如果你不喜欢用英文表达，或者你想要用另外一个名字来描述 column，你可以用 rename 来重新命名。

In [15]:
grouped["评价"].agg(
    [np.sum, np.mean, np.std]
).rename(columns={
    "sum": "合", 
    "mean": "均值", 
    "std": "标准差"
})

,合,均值,标准差
人,,,
小明,129,64.5,10.606602
小红,298,74.5,20.108042


### 总结
我以前在用 Excel 的时候（查 log），经常会用分组来筛选观看数据，现在如果我有了固定的需求， 我完全可以将这些手动 Excel 查 log 的行为自动化到 Python 让它自动帮我完成。你在有同样需求的情况下，也要记得考虑能否用 Pandas 来帮你减负哦。